In [35]:
import pandas as pd
import numpy as np

### Create table to upload to Gaia archive service

In [36]:
clusters = ["ngc2682", "ngc2168","ngc6774","pleiades","praesepe"]

In [37]:
for cluster in clusters:
    df1 = pd.read_csv("{}_topcat.csv".format(cluster))

    dd = df1.loc[~df1.e_bp_min_rp_val_Gaia.isnull(),["source_id_Gaia","e_bp_min_rp_val_Gaia"]].drop_duplicates()

    dd["source_id_Gaia"] = dd["source_id_Gaia"].apply(lambda x: "{:.18f}".format(x).split(".")[0])

    dd.to_csv("{}_gaiaarchive.csv".format(cluster), index=False)

### Now go query for upper and lower percentile of excinction correction

This is the ADQL query for _eilin_ and NGC 2682:

    SELECT source_id, e_bp_min_rp_percentile_lower, e_bp_min_rp_percentile_upper
    FROM gaiadr2.gaia_source
    JOIN user_eilin.ngc2682_topcat as tab
    ON gaiadr2.gaia_source.source_id = tab.source_id_gaia
    
### Next, merge the results back to the topcat table

In [43]:
for cluster in clusters:
    df1 = pd.read_csv("{}_gaiaarchive_results.csv".format(cluster))
    df2 = pd.read_csv("{}_topcat.csv".format(cluster))

    df = df2.merge(df1, how="left", left_on="source_id_Gaia", right_on="source_id")
    del df["source_id"]
    assert df.shape[0] == df2.shape[0]
    assert df.shape[1] == df2.shape[1] + 2
    df.to_csv("{}_topcat_w_extinction_percentiles.csv".format(cluster), index=False)